In [2]:
injection_fpath = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Staging_Lakehouse5.Lakehouse/Files"
bronze_fpath = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Bronze_Lakehouse6.Lakehouse/Files"
silver_fpath = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Silver_Lakehouse5.Lakehouse/Files"
gold_fpath = r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Gold_Lakehouse5.Lakehouse/Files"
silver_delta_path=r"abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Silver_Lakehouse5.Lakehouse/Tables"

StatementMeta(, a0ca0633-8753-44bd-b946-79a8b8d815ba, 3, Finished, Available, Finished)

In [3]:
hr_data_silver = spark.read.format("delta").load( silver_fpath)
hr_data_silver.printSchema()
hr_data_silver.show(5) 

StatementMeta(, a0ca0633-8753-44bd-b946-79a8b8d815ba, 4, Finished, Available, Finished)

root
 |-- employee_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- birthdate: date (nullable = true)
 |-- gender: string (nullable = true)
 |-- race: string (nullable = true)
 |-- department: string (nullable = true)
 |-- jobtitle: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- termdate: timestamp (nullable = true)
 |-- location_city: string (nullable = true)
 |-- location_state: string (nullable = true)

+-----------+----------+---------+----------+--------------+--------------------+--------------------+--------------------+--------+----------+-------------------+-------------+--------------+
|employee_id|first_name|last_name| birthdate|        gender|                race|          department|            jobtitle|location| hire_date|           termdate|location_city|location_state|
+-----------+----------+---------+----------+--------------+-------------

In [4]:
dim_department = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Gold_Lakehouse5.Lakehouse/Tables/DimDepartment" )
dim_location = spark.read.format("delta").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/Gold_Lakehouse5.Lakehouse/Tables/DimLocation")

# Join to get department_id and location_id
joined_df = hr_data_silver \
    .join(dim_department, on="department", how="left") \
    .join(dim_location, on=["location_city", "location_state"], how="left")

StatementMeta(, a0ca0633-8753-44bd-b946-79a8b8d815ba, 5, Finished, Available, Finished)

In [5]:
from pyspark.sql.functions import col, count, avg, sum, when, round, concat_ws, concat, lit


from pyspark.sql.functions import col, floor, datediff, current_date

joined_df = joined_df.withColumn(
    "age",
    floor(datediff(current_date(), col("birthdate")) / 365.25)
)


fact_df = joined_df.groupBy("department_id", "location_id").agg(
    count("employee_id").alias("total_employees"),
    round(avg("age"), 1).alias("avg_age"),

    concat_ws(", ",
        concat(lit("Male: "), round(100 * sum(when(col("gender") == "Male", 1).otherwise(0)) / count("*"), 1), lit("%")),
        concat(lit("Female: "), round(100 * sum(when(col("gender") == "Female", 1).otherwise(0)) / count("*"), 1), lit("%")),
        concat(lit("Other: "), round(100 * sum(when(col("gender") == "Other", 1).otherwise(0)) / count("*"), 1), lit("%"))
    ).alias("gender_distribution"),

    round(100 * sum(when(col("termdate").isNotNull(), 1).otherwise(0)) / count("employee_id"), 2).alias("turnover_rate")
)

StatementMeta(, a0ca0633-8753-44bd-b946-79a8b8d815ba, 6, Finished, Available, Finished)

In [6]:
from pyspark.sql.functions import col, count, avg, sum, when, round, concat_ws, concat, lit


from pyspark.sql.functions import col, floor, datediff, current_date

joined_df = joined_df.withColumn(
    "age",
    floor(datediff(current_date(), col("birthdate")) / 365.25)
)


fact_df = joined_df.groupBy("department_id", "location_id").agg(
    count("employee_id").alias("total_employees"),
    round(avg("age"), 1).alias("avg_age"),

    concat_ws(", ",
        concat(lit("Male: "), round(100 * sum(when(col("gender") == "Male", 1).otherwise(0)) / count("*"), 1), lit("%")),
        concat(lit("Female: "), round(100 * sum(when(col("gender") == "Female", 1).otherwise(0)) / count("*"), 1), lit("%")),
        concat(lit("Other: "), round(100 * sum(when(col("gender") == "Other", 1).otherwise(0)) / count("*"), 1), lit("%"))
    ).alias("gender_distribution"),

    round(100 * sum(when(col("termdate").isNotNull(), 1).otherwise(0)) / count("employee_id"), 2).alias("turnover_rate")
)

StatementMeta(, a0ca0633-8753-44bd-b946-79a8b8d815ba, 7, Finished, Available, Finished)

In [7]:
from pyspark.sql.functions import monotonically_increasing_id

fact_df = fact_df.withColumn("fact_id", monotonically_increasing_id())
fact_df.printSchema()

StatementMeta(, a0ca0633-8753-44bd-b946-79a8b8d815ba, 8, Finished, Available, Finished)

root
 |-- department_id: integer (nullable = true)
 |-- location_id: integer (nullable = true)
 |-- total_employees: long (nullable = false)
 |-- avg_age: double (nullable = true)
 |-- gender_distribution: string (nullable = false)
 |-- turnover_rate: double (nullable = true)
 |-- fact_id: long (nullable = false)



In [8]:
from pyspark.sql.functions import expr, when, col, count, avg, round, sum, lower, trim, monotonically_increasing_id

# Clean gender values
joined_df = joined_df.withColumn("gender", lower(trim(col("gender"))))

# Aggregation logic
fact_df = joined_df.groupBy("department_id", "location_id").agg(
    count("*").alias("total_employees"),
    round(avg("age"), 1).alias("avg_age"),
    
    expr("""
     concat_ws(', ',
        concat('Male: ', ROUND(100 * sum(CASE WHEN gender = 'm' THEN 1 ELSE 0 END) / count(*), 1), '%'),
        concat('Female: ', ROUND(100 * sum(CASE WHEN gender = 'fm' THEN 1 ELSE 0 END) / count(*), 1), '%'),
        concat('Other: ', ROUND(100 * sum(CASE WHEN gender = 'non-conforming' THEN 1 ELSE 0 END) / count(*), 1), '%')
    )
    """).alias("gender_distribution"),

    round(100 * sum(when(col("termdate").isNotNull(), 1).otherwise(0)) / count("*"), 2).alias("turnover_rate")
)

# Add IDs
fact_df = fact_df.withColumn("fact_id", monotonically_increasing_id() + 1)
fact_df = fact_df.withColumn("employee_id", monotonically_increasing_id() + 101)

# Final column order
fact_df_final = fact_df.select(
    "fact_id",
    "employee_id",
    "location_id",
    "department_id",
    "total_employees",
    "avg_age",
    "gender_distribution",
    "turnover_rate"
)

display(fact_df_final)

StatementMeta(, a0ca0633-8753-44bd-b946-79a8b8d815ba, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a94e7c0d-aecf-44fd-a85d-ee1a103855ff)